In [1]:
import os
import numpy as np
import random
import glob
from datetime import datetime
import logging
import sys
import tensorflow as tf
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.layers import Dense, Input, Concatenate
from tensorflow.keras import callbacks, optimizers
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.mixed_precision import set_global_policy
from tensorflow.keras.backend import clear_session
import gc
from sklearn.model_selection import train_test_split
from scipy.stats import kendalltau, spearmanr
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, top_k_accuracy_score, r2_score, mean_squared_error, mean_absolute_error

2024-12-25 20:01:09.465173: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-12-25 20:01:09.579924: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-12-25 20:01:09.609526: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-12-25 20:01:09.815319: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-12-25 20:01:11.146773: W tensorflow/compiler/tf2

In [2]:
# Limpa a sessão antes de construir o modelo
clear_session()

# Configurações de GPU
gpu_devices = tf.config.experimental.list_physical_devices('GPU')
for device in gpu_devices:
    tf.config.experimental.set_memory_growth(device, True)
os.environ['TF_GPU_ALLOCATOR'] = 'cuda_malloc_async'

# Define a política de precisão mista
set_global_policy('mixed_float16')

I0000 00:00:1735167674.658867    4301 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1735167674.886538    4301 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1735167674.886653    4301 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.


In [3]:
output_path = "modelos" #caminha onde é salvo o modelo
files_per_batch = 1000
labeled_data_dir = "train_graphs" #caminho dos dados para treinar o modelo
param_v_a_1 = [4, 3, 2] #camadas compartilhadas rede bound
param_v_a_2 = [3, 2, 2] #camadas ocultas
param_p_a_1 = [6, 4, 3] #camadas compartilhadas rede brach
param_p_a_2 = [9, 6, 2] #camadas ocultas
param_p_b = 250 #batch size
param_v_b =  250
param_p_l = 0.001 #taxa de aprendizado
param_v_l = 0.001
tam = 120

if not os.path.exists(output_path):
    os.makedirs(output_path)

In [4]:
# Pegando todos os arquivos do diretório
files = sorted([os.path.basename(ii) for ii in glob.glob(f"{labeled_data_dir}/*.dimacs")])
files_treino, files_teste = train_test_split(files, test_size=0.1, random_state=42)
files_t, files_v = train_test_split(files_treino, test_size=0.2, random_state=42)

In [5]:
def parse_file_pointer(fp):
    lines = [ll.strip() for ll in fp]
    ii = 0
    labels = []
    res = []
    cli = []
    numLinhas = 0
    while ii < len(lines):
        line = lines[ii]
        #contando o numero de vertices do grafo
        if "cliqueatual" not in line:
            ii += 1
            numLinhas += 1
            continue

        #pegando a clique atual
        if ii+1 >= len(lines):
            break
        line = line[3:]
        spritado = line.split()
        clique = [int(elem) for elem in spritado[1:]]
        if(numLinhas < tam):
            dif = tam - numLinhas
            clique.extend([0]*dif)
        cli.append(clique)

        #criando o vetor de movimento
        line = lines[ii+1]
        sp = line.split()
        mv = int(sp[-1])
        label = [0] * tam
        label[mv] = 1
        labels.append(label)

        #lendo o grafo
        cells = []
        for tt in range(numLinhas, 0, -1):
            cell_line = lines[ii - tt][3:]
            cells.extend([int(float(cc)) for cc in cell_line.split(", ")])
            if(numLinhas < tam):
                dif = tam - numLinhas
                cells.extend([0]*dif)
        while len(cells) < tam * tam:
            cells.extend([0]*tam)
        res.append(cells)
        ii += (numLinhas+2)
    labels_v = list(range(len(labels),0, -1))
    return (res, cli, labels, labels_v)

In [6]:
def estruturar_entrada(batch_input, batch_labels):
    # Dividir a entrada em uma lista de 151 tensores de forma (batch_size, tam)
    batch_input_list = [batch_input[:, i, :] for i in range(batch_input.shape[1])]
            
    # Converter para tensores do TensorFlow
    x_batch = [tf.convert_to_tensor(tensor, dtype=tf.float32) for tensor in batch_input_list]
    input_dict = {f'input_{i}': tensor for i, tensor in enumerate(x_batch)}
    y_batch = np.array(batch_labels)
    return input_dict, y_batch

In [7]:
def combinar_entrada(res, clique, labels, remaining_batch_input=[], remaining_batch_labels=[]):
    combined_input = np.array([np.hsplit(np.concatenate([res[i], clique[i]]), tam + 1) for i in range(len(clique))])
    if len(remaining_batch_input) != 0:
        combined_input = np.concatenate((remaining_batch_input, combined_input), axis=0)
        labels = remaining_batch_labels + labels
    return combined_input, labels

In [8]:
def parse_dir(files):
    res = []
    cli = []
    labels = []
    labels_v = []
    random.seed(42)
    random.shuffle(files)
    random.seed()
    for ff in files:
        with open(os.path.join(labeled_data_dir,ff), 'r') as fp:
            rr, cc, ll, ll_v = parse_file_pointer(fp)
            res.extend(rr)
            cli.extend(cc)
            labels.extend(ll)
            labels_v.extend(ll_v)
    return res, cli, labels, labels_v

In [9]:
def ler_arquivos(batch_files, value):
    res, clique, labels, labels_v = parse_dir(batch_files)
    if value:
        labels = labels_v
    return res, clique, labels

In [10]:
def data_generator(batch_size, value):
    # Gerador de treinamento
    def create_generator(file_list):
        def generator():
            idx = 0
            remaining_batch_input = remaining_batch_labels = []
            while idx < len(file_list):
                batch_files = file_list[idx: idx + files_per_batch]
                res, clique, labels = ler_arquivos(batch_files, value)
                combined_input, labels = combinar_entrada(res, clique, labels, remaining_batch_input, remaining_batch_labels)
                remaining_batch_input = remaining_batch_labels = []
                num_samples = len(combined_input)
                num_batches = num_samples // batch_size
                for batch_idx in range(num_batches):
                    batch_input = combined_input[batch_idx * batch_size: (batch_idx + 1) * batch_size]
                    batch_labels = labels[batch_idx * batch_size: (batch_idx + 1) * batch_size]
                    x_batch, y_batch = estruturar_entrada(batch_input, batch_labels)
                    yield x_batch, y_batch
                idx += files_per_batch
                remaining_samples = num_samples % batch_size
                if remaining_samples > 0:
                    remaining_batch_input = combined_input[-remaining_samples:]
                    remaining_batch_labels = labels[-remaining_samples:]
        return generator
    
    #ler o dataset de validação todo de uma vez ao invés de usar um gerador pra ele
    res, clique, labels = ler_arquivos(files_v, value)
    combined_input, labels = combinar_entrada(res, clique, labels)
    x_value, y_value = estruturar_entrada(combined_input, labels)
    
    return create_generator(files_t)(), x_value, y_value

In [11]:
def output_signature_type_branch(batch_size):
    output_signature=(
            {f"input_{i}": tf.TensorSpec(shape=(batch_size, tam), dtype=tf.float32) for i in range(tam + 1)},  # Entradas
            tf.TensorSpec(shape=(batch_size, tam), dtype=tf.float32)  # Rótulos
    )
    return output_signature

def output_signature_type_bound(batch_size):
    output_signature=(
            {f"input_{i}": tf.TensorSpec(shape=(batch_size, tam), dtype=tf.float32) for i in range(tam + 1)},  # Entradas
            tf.TensorSpec(shape=(batch_size,), dtype=tf.float32)  # Rótulos
    )
    return output_signature

In [12]:
class printbatch(callbacks.Callback):
    def on_epoch_begin(self, epoch, logs={}):
        logging.info("Epoch: "+ str(epoch))
    def on_epoch_end(self, epoch, logs={}):
        logging.info(logs)

In [14]:
#treinar rede branch
#verificar se já tem um modelo compilado para treinar
initial_epoch = 0
checkpoint_dir = os.path.join(output_path, "checkpoints", f"dnn_model_{tam}")
os.makedirs(checkpoint_dir, exist_ok=True)
latest_checkpoint = tf.train.latest_checkpoint(checkpoint_dir)
checkpoint_path = os.path.join(checkpoint_dir, "checkpoint_epoch_{epoch:04d}.keras")

if latest_checkpoint:
    print(f"Carregando modelo do checkpoint: {latest_checkpoint}")
    model = tf.keras.models.load_model(latest_checkpoint)
    # Extraindo a última época treinada
    initial_epoch = int(latest_checkpoint.split("_epoch_")[1].split(".keras")[0])

elif os.path.exists(os.path.join(checkpoint_dir, 'compiled_model.keras')):
    print("Carregando modelo compilado salvo...")
    model = tf.keras.models.load_model(os.path.join(checkpoint_dir, 'compiled_model.keras'))

else:
    # Definir camadas de entrada
    inputArray = [Input(shape=(tam,), name=f"input_{i}") for i in range(tam + 1)]

    # Camadas densas compartilhadas
    layer = inputArray
    for i in range(len(param_p_a_1)):
        shared_dense = Dense(tam * param_p_a_1[i], activation='relu')
        layer = [shared_dense(l) for l in layer]

    # Concatenar os vetores processados
    merged_vector = Concatenate(axis=-1)(layer)

    #camadas internas
    layer = merged_vector
    for i in range(len(param_p_a_2)):
        layer = Dense(tam*(tam+1)*param_p_a_2[i],activation='relu')(layer)

    #camada de saida
    output_layer = Dense(tam, activation='softmax')(layer)

    #compilar modelo
    model = Model(inputs=inputArray, outputs=output_layer)
    adam = optimizers.Adam(learning_rate=param_p_l)
    model.compile(optimizer=adam, loss='categorical_crossentropy', metrics=['accuracy'])
    
# Criar os datasets de treino e validação
train_generator_func, val_x, val_y = data_generator(param_p_b, False)
train_dataset = tf.data.Dataset.from_generator(lambda : train_generator_func, output_signature=output_signature_type_branch(param_p_b))
train_dataset = train_dataset.prefetch(tf.data.AUTOTUNE)
    
now = datetime.now()
# Treinar modelo
print("treinamento rede branch iniciado")
model.fit(
    train_dataset,
    epochs=1000,
    initial_epoch=initial_epoch,
    verbose=1,
    validation_data=(val_x, val_y),
    callbacks=[
        printbatch(),
        EarlyStopping(monitor='val_loss', patience=50, verbose=0),
        ModelCheckpoint(filepath=checkpoint_path, monitor='val_loss', save_best_only=False, save_weights_only=False)
    ]
)
final_model_dnn_path = os.path.join(output_path, f"dnn_model_{tam}_{now.day}.{now.month}.{now.year}.keras")
model.save(final_model_dnn_path)
del model, val_x, val_y, train_dataset, train_generator_func, inputArray, merged_vector, layer, output_layer
print("rede 1 treinada")
clear_session()
gc.collect()

In [ ]:
#treinar rede bound
checkpoint_dir = os.path.join(output_path, "checkpoints", f"dnn_value_model_{tam}")
os.makedirs(checkpoint_dir, exist_ok=True)
checkpoint_path = os.path.join(checkpoint_dir, "checkpoint_epoch_{epoch:04d}.keras")

# Verificar se existe um checkpoint salvo
initial_epoch = 0
latest_checkpoint = tf.train.latest_checkpoint(checkpoint_dir)
if latest_checkpoint:
    print(f"Carregando modelo do checkpoint: {latest_checkpoint}")
    model = tf.keras.models.load_model(latest_checkpoint)
    # Extraindo a última época treinada
    initial_epoch = int(latest_checkpoint.split("_epoch_")[1].split(".keras")[0])

elif os.path.exists(os.path.join(checkpoint_dir, 'compiled_model.keras')):
    print("Carregando modelo compilado salvo...")
    model = tf.keras.models.load_model(os.path.join(checkpoint_dir, 'compiled_model.keras'))

else: 
    # Definir camadas de entrada
    inputArray = [Input(shape=(tam,), name=f"input_{i}") for i in range(tam + 1)]

    # Camadas densas compartilhadas
    layer = inputArray
    for i in range(len(param_v_a_1)):
        shared_dense = Dense(tam * param_v_a_1[i], activation='relu')
        layer = [shared_dense(l) for l in layer]

    # Concatenar os vetores processados
    merged_vector = Concatenate(axis=-1)(layer)

    #camadas internas
    layer = merged_vector
    for i in range(len(param_v_a_2)):
        layer = Dense(tam*(tam+1)*param_v_a_2[i],activation='relu')(layer)

    #camada de saida
    output_layer = Dense(1)(layer)

    #compilar modelo
    model = Model(inputs=inputArray, outputs=output_layer)
    adam = optimizers.Adam(learning_rate=param_v_l)
    model.compile(optimizer=adam, loss='mse', metrics=['mae'])

# Criar os datasets de treino e validação
train_generator_func, val_x, val_y = data_generator(param_v_b, True)
train_dataset = tf.data.Dataset.from_generator(lambda : train_generator_func, output_signature=output_signature_type_bound(param_v_b))
train_dataset = train_dataset.prefetch(tf.data.AUTOTUNE)

now = datetime.now()
# Treinar modelo
print("treinamento rede bound iniciado")
model.fit(
    train_dataset,
    epochs=1000,
    initial_epoch=initial_epoch,
    verbose=1,
    validation_data=(val_x, val_y),
    callbacks=[
        printbatch(),
        EarlyStopping(monitor='val_loss', patience=50, verbose=0),
        ModelCheckpoint(filepath=checkpoint_path, monitor='val_loss', save_best_only=True, save_weights_only=False)
    ]
)
final_model_value_path = os.path.join(output_path, f"dnn_value_model_{tam}_{now.day}.{now.month}.{now.year}.keras")
model.save(final_model_value_path)
del model, val_x, val_y, train_dataset, train_generator_func, inputArray, merged_vector, layer, output_layer
print("rede 2 treinada")
clear_session()
gc.collect()

In [ ]:
# Acurácia
'''def calcular_acuracia(true_values, predictions):
    return accuracy_score(true_values, predictions)

# Precisão
def calcular_precisao(true_values, predictions, average='binary'):
    return precision_score(true_values, predictions, average=average)

# Recall
def calcular_recall(true_values, predictions, average='binary'):
    return recall_score(true_values, predictions, average=average)

# F1-Score
def calcular_f1(true_values, predictions, average='binary'):
    return f1_score(true_values, predictions, average=average)

# Top-K Accuracy
def calcular_top_k_accuracy(true_values, predictions_probs, k=3):
    return top_k_accuracy_score(true_values, predictions_probs, k=k)

def calcular_mae(true_values, predictions):
    return mean_absolute_error(true_values, predictions)

def calcular_mse(true_values, predictions):
    return mean_squared_error(true_values, predictions)

def calcular_r2(true_values, predictions):
    return r2_score(true_values, predictions)

def calcular_kendalltau(true_values, predictions):
    return kendalltau(true_values, predictions)

def calcular_spearmanr(true_values, predictions):
    return spearmanr(true_values, predictions)'''

In [ ]:
# Função para testar o modelo
'''def testar_modelo(modelo, files_test, usar_labels_value):
    res, clique, labels = ler_arquivos(files_test, usar_labels_value)
    combined_input, labels = combinar_entrada(res, clique, labels)
    x_test, y_test = estruturar_entrada(combined_input, labels)
    # Prever usando o modelo
    previsoes = modelo.predict(x_test)
    # Calcular métricas
    acuracia = calcular_acuracia(y_test, previsoes)
    print(f"Acurácia do modelo: {acuracia:.2f}")
    precisao = calcular_precisao(y_test, previsoes)
    print(f"Precisão do modelo: {precisao:.2f}")
    recall = calcular_recall(y_test, previsoes)
    print(f"Recall do modelo: {recall:.2f}")
    f1 = calcular_f1(y_test, previsoes)
    print(f"F1 do modelo: {f1:.2f}")
    top_k = calcular_top_k_accuracy(y_test, previsoes)
    print(f"Top K do modelo: {top_k:.2f}")
    mae = calcular_mae(y_test, previsoes)
    print(f"MAE do modelo: {mae:.2f}")
    mse = calcular_mse(y_test, previsoes)
    print(f"MSE do modelo: {mse:.2f}")
    r2 = calcular_r2(y_test, previsoes)
    print(f"R2 do modelo: {r2:.2f}")
    # Correlação de Kendall
    tau, _ = calcular_kendalltau(y_test, previsoes)
    print(f"Kendall's Tau: {tau:.4f}")
    # Correlação de Spearman
    rho, _ = calcular_spearmanr(y_test, previsoes)
    print(f"Spearman Correlation: {rho:.4f}")'''

In [ ]:
'''print("Carregando o modelo branch...")
modelo_branch = load_model(final_model_dnn_path)
print("Testando o modelo branch...")
testar_modelo(modelo_branch, files_teste, False)'''

In [ ]:
'''print("Carregando o modelo bound...")
modelo_bound = load_model(final_model_value_path)
print("Testando o modelo bound...")
testar_modelo(modelo_bound, files_teste, True)'''